### Redes neuronales Feed Forward

Las redes "feed forward" o $\text{FFN}$ de ahora en adelante, son tipo de redes neuronales en las que los datos entran por la capa de entrada y se transmiten en una única dirección hacia la capa de salida. El tipo de red neuronal mas común del tipo $\text{FFN}$ son los perceptrones multicapa, que consisten en una serie de capas de neuronas, donde cada neurona de una capa está conectada con todas las neuronas de la capa siguiente.

En este trabajo estaremos comparando el rendimiento de redes $\text{FFN}$ del tipo autoencoder, ya que es asi como se utilizan en los transformers para aprender a transformar las secuencias de una representacion en el espacio de embeddings en otra entre las capas de la red, y a que queremos evaluarlas en conjunto con los metodos de normalización disponibles. 

#### Elección del tamaño de la capa oculta.

Sean:

- $d$ la dimensión del modelo.
- $h$ el tamaño de la capa oculta de la red tipo $\text{MLP}$.
- $h'$ el tamaño de la capa oculta de la red tipo $\text{GLU}$.

El numero de parámetros de la red tipo $\text{MLP}$ sera de $dh + hd = 2dh $, sin bias y $dh + hd + h = (2d + 1)h$ con bias. Notemos que ignoramos el termino de bias en la ultima capa debido a que este es incompatible con los metodos normalización utilizados actualmente.

In [ ]:
from torch import Tensor
from torch.nn import Module, Linear, Sequential
from torch.nn import ReLU
from torch.nn import Dropout

class MLP(Module):
    def __init__(self, model_dimension: int, hidden_dimension: int, bias: bool = True, p: float = 0.5):
        super().__init__()
        self.input_layer = Linear(model_dimension, hidden_dimension, bias=bias)
        self.activation = ReLU()
        self.dropout = Dropout(p)
        self.output_layer = Linear(hidden_dimension, model_dimension, bias=False)
       
    def forward(self, input: Tensor) -> Tensor:
        output = self.activation(self.input_layer(input))
        output = self.dropout(output)
        return self.output_layer(output)

class GLU(Module):
    def __init__(self, model_dimension: int, hidden_dimension: int, bias: bool = True):
        super().__init__()
        if not hidden_dimension % 3 == 0:
            print("Warning: hidden_dimension should be a multiple of 3 for having the same number of parameters as a regular MLP")
        self.hidden_dimension = (hidden_dimension * 2) // 3
        self.gate_dimension = hidden_dimension 

        self.activation = ReLU()
        self.input_layer = Linear(model_dimension, self.hidden_dimension, bias=bias)
        self.output_layer = Linear(self.hidden_dimension, hidden_dimension, bias=False)
        self.gate_layer = Linear(model_dimension, self.hidden_dimension, bias=False)
       
    def forward(self, input: Tensor) -> Tensor:
        output = self.activation(self.input_layer(input)) * self.gate_layer(input)
        return self.output_layer(output)

In [ ]:
HIDDEN_SIZE_S = 384
HIDDEN_SIZE_M = 576 